In [ ]:
pip install pandas scikit-learn nltk


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
# Try different encodings if UTF-8 fails
try:
    df = pd.read_csv('spam.csv', encoding='utf-8', usecols=['v1', 'v2'])
except UnicodeDecodeError:
    df = pd.read_csv('spam.csv', encoding='latin1', usecols=['v1', 'v2'])

# Assuming you want to read the first row of data
row_index = 0

# Extract data from v1 and v2 columns
data_v1 = df.at[row_index, 'v1']
data_v2 = df.at[row_index, 'v2']

# Printing data for demonstration
print(data_v1, "\t", data_v2)


ham 	 Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


In [ ]:
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Load your data into a DataFrame
# Try different encodings if UTF-8 fails
try:
    df = pd.read_csv('spam.csv', encoding='utf-8', usecols=['v1', 'v2'])
except UnicodeDecodeError:
    df = pd.read_csv('spam.csv', encoding='latin1', usecols=['v1', 'v2'])

# Define stopwords
stop_words = set(stopwords.words('english'))

# Preprocess the text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords and split into words
    words = [word for word in text.split() if word not in stop_words]
    # Join the words back into a single string
    text = ' '.join(words)
    return text

# Apply preprocessing to your text data (replace 'v2' with your actual text column name)
df['v2'] = df['v2'].apply(preprocess_text)

# Example: Print the preprocessed DataFrame
print(df.head())


     v1                                                 v2
0   ham  go jurong point crazy available bugis n great ...
1   ham                            ok lar joking wif u oni
2  spam  free entry 2 wkly comp win fa cup final tkts 2...
3   ham                u dun say early hor u c already say
4   ham        nah dont think goes usf lives around though


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)

# Transform the text data
X = tfidf.fit_transform(df['v2']).toarray()
y = df['v1']

print(X.shape)


(5572, 5000)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the models
log_reg = LogisticRegression()
naive_bayes = MultinomialNB()
svc = SVC()

# Train the models
log_reg.fit(X_train, y_train)
naive_bayes.fit(X_train, y_train)
svc.fit(X_train, y_train)


SVC()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Predict and evaluate
models = {'Logistic Regression': log_reg, 'Naive Bayes': naive_bayes, 'SVM': svc}
for model_name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{model_name} Accuracy: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))


Logistic Regression Accuracy: 0.9434977578475336
              precision    recall  f1-score   support

         ham       0.94      1.00      0.97       965
        spam       0.96      0.61      0.74       150

    accuracy                           0.94      1115
   macro avg       0.95      0.80      0.86      1115
weighted avg       0.94      0.94      0.94      1115

Naive Bayes Accuracy: 0.9739910313901345
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       965
        spam       1.00      0.81      0.89       150

    accuracy                           0.97      1115
   macro avg       0.99      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

SVM Accuracy: 0.9739910313901345
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       965
        spam       0.99      0.81      0.89       150

    accuracy                           0.97      1115
   ma

In [ ]:
import joblib

# Save the best model (e.g., Logistic Regression)
joblib.dump(log_reg, 'spam_classifier.pkl')

# Load the model
model = joblib.load('spam_classifier.pkl')

# Predict with the loaded model
y_pred_loaded = model.predict(X_test)
print(f"Loaded Model Accuracy: {accuracy_score(y_test, y_pred_loaded)}")


Loaded Model Accuracy: 0.9434977578475336


In [56]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# Further Model Evaluation
for model_name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{model_name} Evaluation:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Precision: {precision_score(y_test, y_pred, pos_label='spam')}")
    print(f"Recall: {recall_score(y_test, y_pred, pos_label='spam')}")
    print(f"F1-score: {f1_score(y_test, y_pred, pos_label='spam')}")
    print(classification_report(y_test, y_pred))

Logistic Regression Evaluation:
Accuracy: 0.9434977578475336
Precision: 0.9578947368421052
Recall: 0.6066666666666667
F1-score: 0.7428571428571428
              precision    recall  f1-score   support

         ham       0.94      1.00      0.97       965
        spam       0.96      0.61      0.74       150

    accuracy                           0.94      1115
   macro avg       0.95      0.80      0.86      1115
weighted avg       0.94      0.94      0.94      1115

Naive Bayes Evaluation:
Accuracy: 0.9739910313901345
Precision: 1.0
Recall: 0.8066666666666666
F1-score: 0.8929889298892989
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       965
        spam       1.00      0.81      0.89       150

    accuracy                           0.97      1115
   macro avg       0.99      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

SVM Evaluation:
Accuracy: 0.9739910313901345
Precision: 0.991869918699187
R

In [ ]:
# Hyperparameter Tuning (example using RandomForestClassifier)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20]
}

rf_classifier = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best parameters:", best_params)
